In [1]:
import numpy as np
import xarray as xr
import os
import glob
from pathlib import Path 
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt
import time
import requests
import s3fs
from os.path import dirname, join
import datetime
import myDate
import netCDF4

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

In [2]:
grids_file_dir = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/')
grids_file_dir_randomnoise = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/')
grids_file_dir_missingdata = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/')
grids_file_dir_orbiterror_2cm = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/')
grids_file_dir_3errors_2cm = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/')
grids_file_dir_beckley = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/')

main_dir = Path('/home/jpluser/ECCO_GMSL/')
output_dir = Path('/home/jpluser/ECCO_GMSL/Data/GMSL/')
fig_dir = Path('/home/jpluser/ECCO_GMSL/Figures/')

# Compute GMSL from simple gridder of Beckley along track data daily files

In [3]:
beckley_grids_files = list(grids_file_dir_beckley.glob('ssha_global_half_deg_*nc'))
beckley_grids_files=np.sort(beckley_grids_files)
print(beckley_grids_files[0:5])
print('...')
print(beckley_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921017.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921027.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921106.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921116.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921126.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171125.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171205.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171215.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171225.n

In [4]:
grids = xr.open_mfdataset(
        paths=beckley_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )

In [5]:
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 920, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-10-17 1992-10-27 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [6]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))

In [7]:
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
weights.shape

(920, 264, 720)

In [8]:
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [9]:
# make new DataArray objects
gmsl_grids_beckley = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids_beckley = gmsl_grids_beckley.assign_coords({'time':grids.time.data})
gmsl_grids_beckley.name = 'gmsl_grids_beckley'
gmsl_grids_beckley

<xarray.DataArray 'gmsl_grids_beckley' (time: 920)> Size: 7kB
array([ 5.40784551e-04, -1.97048707e-03, -4.17616933e-03, -5.14092416e-03,
       -5.40129909e-03, -9.09074737e-03, -1.38629680e-02, -1.61661639e-02,
       -1.82103633e-02, -2.00380847e-02, -2.21026697e-02, -2.21912848e-02,
       -2.05491205e-02, -1.80682500e-02, -1.75653744e-02, -1.79529609e-02,
       -2.01328771e-02, -2.12346179e-02, -1.99625716e-02, -1.88330686e-02,
       -2.09599672e-02, -2.14706764e-02, -2.18069356e-02, -2.21669449e-02,
       -2.01728981e-02, -1.73505041e-02, -1.71980089e-02, -1.62088475e-02,
       -1.84356199e-02, -1.62956117e-02, -1.40167992e-02, -1.14383288e-02,
       -1.02417582e-02, -1.11739665e-02, -1.00442089e-02, -9.07789052e-03,
       -6.13102950e-03, -6.14556384e-03, -6.74117835e-03, -9.52057695e-03,
       -1.07175447e-02, -1.06322603e-02, -8.36609345e-03, -6.94307041e-03,
       -9.92204007e-03, -1.25471399e-02, -1.16615474e-02, -1.32627739e-02,
       -1.58953560e-02, -1.68576500e-02, -1.38449509e-02, -1.33039811e-02,
       -1.32689327e-02, -1.34552302e-02, -1.34251617e-02, -1.40291253e-02,
       -1.64739395e-02, -1.80117291e-02, -1.96335051e-02, -1.81257858e-02,
       -1.59542411e-02, -1.56311187e-02, -1.72069364e-02, -1.65453203e-02,
       -1.55168251e-02, -1.43299526e-02, -1.09456182e-02, -8.88100537e-03,
       -7.74835109e-03, -6.54840819e-03, -5.45205885e-03, -4.52613441e-03,
       -3.56099878e-03, -1.52044629e-03, -1.61289040e-03, -3.04205841e-03,
       -4.76644693e-03, -5.86892834e-03, -5.33823764e-03, -2.58716141e-03,
...
        6.45168367e-02,  6.23568122e-02,  5.98356307e-02,  6.08397890e-02,
        6.03785521e-02,  5.98044864e-02,  5.61751139e-02,  5.48386063e-02,
        5.77193405e-02,  5.76747211e-02,  5.77891758e-02,  5.68248769e-02,
        5.59027839e-02,  5.49047948e-02,  5.36780123e-02,  5.34112939e-02,
        5.13068477e-02,  4.97151980e-02,  4.98338734e-02,  4.86781973e-02,
        4.82384035e-02,  4.84028743e-02,  4.93605364e-02,  5.07823621e-02,
        5.17131040e-02,  5.19952964e-02,  5.11904650e-02,  5.12687966e-02,
        5.14532375e-02,  5.23217248e-02,  5.37453275e-02,  5.81886641e-02,
        5.91521747e-02,  6.04512325e-02,  5.88639037e-02,  5.98902615e-02,
        6.21599793e-02,  6.27013523e-02,  6.27563922e-02,  6.42124723e-02,
        6.19524881e-02,  6.02427187e-02,  5.84901996e-02,  5.58139611e-02,
        5.49361236e-02,  5.51734987e-02,  5.24541929e-02,  5.09659131e-02,
        5.10613084e-02,  5.20060741e-02,  5.21091134e-02,  5.15780140e-02,
        5.17571307e-02,  5.16277934e-02,  5.09615317e-02,  4.97612847e-02,
        4.80563522e-02,  4.71638838e-02,  4.76406885e-02,  4.83147303e-02,
        4.98514906e-02,  5.03653010e-02,  5.00100903e-02,  5.20560652e-02,
        5.39846908e-02,  5.67028605e-02,  5.71091336e-02,  5.82973370e-02,
        6.05541496e-02,  6.19264908e-02,  6.11148843e-02,  6.22416357e-02,
        6.34231759e-02,  6.43239879e-02,  6.40277304e-02,  6.46388451e-02,
        6.42018504e-02,  6.44614976e-02,  6.26402958e-02,  5.95164039e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-10-17 1992-10-27 ... 2017-12-25

In [10]:
fname = output_dir / ('GMSL_Beckley_dailyfiles_grids.nc')
gmsl_grids_beckley.to_netcdf(fname)

# Compute GMSL from ECCO synthetic grids with simple weighing

### No error

In [11]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2018-01-04.nc')]


In [12]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [13]:
grids=grids.where(np.array(grids.counts)>=100)

In [14]:
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [15]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [16]:
# make new DataArray objects
gmsl_grids = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids = gmsl_grids.assign_coords({'time':grids.time})
gmsl_grids.name = 'gmsl_grids'
gmsl_grids

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.36559075e-02, -3.15734014e-02, -2.94833375e-02, -3.00416273e-02,
       -3.11876765e-02, -3.29730372e-02, -3.12643644e-02, -3.27532860e-02,
       -3.31629530e-02, -3.88991561e-02, -4.08632415e-02, -4.04851514e-02,
       -4.09820391e-02, -4.24949987e-02, -4.28519874e-02, -4.02127380e-02,
       -3.99657250e-02, -3.84151946e-02, -4.09518685e-02, -3.88663286e-02,
       -3.80046876e-02, -3.73287441e-02, -3.86964271e-02, -3.86234508e-02,
       -3.72344515e-02, -3.70624164e-02, -3.80155304e-02, -3.76083903e-02,
       -3.92600687e-02, -3.79097133e-02, -3.67560938e-02, -3.74289726e-02,
       -3.47385105e-02, -3.39390883e-02, -3.23579176e-02, -3.21599779e-02,
       -3.06515419e-02, -2.84438732e-02, -2.76587758e-02, -2.77995240e-02,
       -2.93538040e-02, -3.01110306e-02, -3.26589968e-02, -3.18998348e-02,
       -2.84424220e-02, -2.69171937e-02, -3.15929112e-02, -3.24506270e-02,
       -3.06481931e-02, -3.42203704e-02, -3.83785655e-02, -3.93057743e-02,
       -3.51508920e-02, -3.33307566e-02, -3.22152689e-02, -3.34335887e-02,
       -3.18224583e-02, -3.33990338e-02, -3.33316702e-02, -3.64033170e-02,
       -3.60441961e-02, -3.57184328e-02, -3.87875862e-02, -3.95911378e-02,
       -4.02223259e-02, -3.92352686e-02, -3.75370141e-02, -3.71225069e-02,
       -3.59458507e-02, -3.57585748e-02, -3.21608195e-02, -3.24446375e-02,
       -2.99393205e-02, -2.97725934e-02, -2.69412485e-02, -2.77069144e-02,
       -2.54326734e-02, -2.63608289e-02, -2.78452190e-02, -2.83351473e-02,
...
        4.68764091e-02,  4.60500311e-02,  4.74359008e-02,  4.49400484e-02,
        4.37936831e-02,  3.97827573e-02,  3.83412771e-02,  4.19647847e-02,
        4.11925999e-02,  3.77726150e-02,  3.81461966e-02,  3.64780245e-02,
        3.81895293e-02,  3.58122454e-02,  3.85917476e-02,  3.64220130e-02,
        3.75142483e-02,  3.70616735e-02,  3.90377390e-02,  3.84751410e-02,
        3.65507909e-02,  3.80483035e-02,  3.72829081e-02,  3.69074991e-02,
        3.70448280e-02,  3.64092984e-02,  3.52751335e-02,  3.85878970e-02,
        3.90531082e-02,  3.98380412e-02,  4.18476358e-02,  4.32232870e-02,
        4.26313710e-02,  4.25291043e-02,  4.40274684e-02,  4.66646959e-02,
        4.63785560e-02,  4.84996505e-02,  4.76826793e-02,  4.90430955e-02,
        4.71887602e-02,  4.68751496e-02,  4.51644803e-02,  4.52320030e-02,
        4.24969349e-02,  3.88052691e-02,  3.69834169e-02,  3.75596118e-02,
        3.72659697e-02,  3.77331149e-02,  3.75831554e-02,  3.76875421e-02,
        3.94179199e-02,  4.04372612e-02,  4.11147190e-02,  4.16326097e-02,
        4.00007157e-02,  3.98980435e-02,  4.25903584e-02,  4.12330753e-02,
        4.17725325e-02,  4.03097253e-02,  4.09283807e-02,  4.32351431e-02,
        4.41061060e-02,  4.65801125e-02,  4.70414970e-02,  4.77456133e-02,
        4.96097840e-02,  4.75282034e-02,  4.88240449e-02,  4.94618449e-02,
        5.26005543e-02,  5.39350413e-02,  5.44330689e-02,  5.66874547e-02,
        5.50303774e-02,  5.58749204e-02,  5.38400634e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Random noise

In [17]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_randomnoise.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-25.n

In [18]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [19]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [20]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [21]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [22]:
# ssh_weighted_mean_randomnoise = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_randomnoise.SSHA

In [23]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_randomnoise=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [24]:
# make new DataArray objects
gmsl_grids_randomnoise = xr.DataArray(ssh_weighted_mean_randomnoise, dims=['time'])
gmsl_grids_randomnoise = gmsl_grids_randomnoise.assign_coords({'time':grids.time})
gmsl_grids_randomnoise.name = 'gmsl_grids'
gmsl_grids_randomnoise

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.37101154e-02, -3.15875817e-02, -2.94421708e-02, -3.00153049e-02,
       -3.12239300e-02, -3.29420963e-02, -3.12947063e-02, -3.27563039e-02,
       -3.32506593e-02, -3.89149269e-02, -4.08561870e-02, -4.04671374e-02,
       -4.08932858e-02, -4.25091748e-02, -4.28614714e-02, -4.01357299e-02,
       -3.99887870e-02, -3.83475775e-02, -4.08962069e-02, -3.88452396e-02,
       -3.79820980e-02, -3.74029390e-02, -3.86855608e-02, -3.86667543e-02,
       -3.72306967e-02, -3.70351494e-02, -3.79932733e-02, -3.76366005e-02,
       -3.92391357e-02, -3.78772234e-02, -3.67863492e-02, -3.73855040e-02,
       -3.48342107e-02, -3.38979422e-02, -3.23943334e-02, -3.22403412e-02,
       -3.06059353e-02, -2.84032206e-02, -2.75946058e-02, -2.77874878e-02,
       -2.92705147e-02, -3.01440378e-02, -3.26637124e-02, -3.19102208e-02,
       -2.84656147e-02, -2.68868521e-02, -3.15868693e-02, -3.23818158e-02,
       -3.06198253e-02, -3.42468367e-02, -3.83005283e-02, -3.93709387e-02,
       -3.51468524e-02, -3.32416614e-02, -3.22574971e-02, -3.34755725e-02,
       -3.17628182e-02, -3.35040504e-02, -3.32794487e-02, -3.64417007e-02,
       -3.60707718e-02, -3.56942744e-02, -3.87957901e-02, -3.97045765e-02,
       -4.01469362e-02, -3.92351713e-02, -3.75693471e-02, -3.71145425e-02,
       -3.59825637e-02, -3.57256759e-02, -3.20679908e-02, -3.24827872e-02,
       -2.99472445e-02, -2.97507698e-02, -2.69023881e-02, -2.77196208e-02,
       -2.55098483e-02, -2.64293750e-02, -2.78804959e-02, -2.82954558e-02,
...
        4.68449630e-02,  4.61655506e-02,  4.74829852e-02,  4.49295425e-02,
        4.37231485e-02,  3.97242773e-02,  3.82805262e-02,  4.19840245e-02,
        4.11344120e-02,  3.77446543e-02,  3.81634021e-02,  3.65066425e-02,
        3.81704742e-02,  3.58624481e-02,  3.85882291e-02,  3.64223681e-02,
        3.75194605e-02,  3.70680579e-02,  3.90467463e-02,  3.84535018e-02,
        3.65723795e-02,  3.80375792e-02,  3.73361650e-02,  3.68950672e-02,
        3.70196523e-02,  3.64667771e-02,  3.53024357e-02,  3.86286175e-02,
        3.90579194e-02,  3.98471808e-02,  4.17877733e-02,  4.32618437e-02,
        4.26101992e-02,  4.25429972e-02,  4.39719478e-02,  4.66184929e-02,
        4.63694014e-02,  4.84511552e-02,  4.76595174e-02,  4.90752920e-02,
        4.71431693e-02,  4.68607714e-02,  4.51638796e-02,  4.51931334e-02,
        4.25277511e-02,  3.87311758e-02,  3.70661529e-02,  3.75990701e-02,
        3.72626463e-02,  3.76120210e-02,  3.75546879e-02,  3.77630737e-02,
        3.94059222e-02,  4.04525138e-02,  4.11600347e-02,  4.16448040e-02,
        4.00069392e-02,  3.99587893e-02,  4.25665330e-02,  4.11900418e-02,
        4.17779100e-02,  4.03133136e-02,  4.09378669e-02,  4.32882113e-02,
        4.40993087e-02,  4.66074300e-02,  4.70093577e-02,  4.77573879e-02,
        4.95055672e-02,  4.75146975e-02,  4.87552908e-02,  4.95124116e-02,
        5.26144209e-02,  5.39655769e-02,  5.44545666e-02,  5.66607596e-02,
        5.50029607e-02,  5.58204954e-02,  5.38912843e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Missing data

In [25]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_missingdata.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-25.n

In [26]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [27]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [28]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [29]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [30]:
# ssh_weighted_mean_missingdata = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_missingdata.SSHA

In [31]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_missingdata=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [32]:
# make new DataArray objects
gmsl_grids_missingdata = xr.DataArray(ssh_weighted_mean_missingdata, dims=['time'])
gmsl_grids_missingdata = gmsl_grids_missingdata.assign_coords({'time':grids.time})
gmsl_grids_missingdata.name = 'gmsl_grids'
gmsl_grids_missingdata

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([            nan,             nan, -2.99814071e-02, -3.02057364e-02,
       -3.14227670e-02, -3.34293599e-02, -3.17877893e-02, -3.35307562e-02,
       -3.25451265e-02, -3.87030157e-02, -4.04545244e-02, -3.99949254e-02,
       -4.04655282e-02, -4.21619555e-02, -4.26121638e-02, -3.98091064e-02,
       -3.97788145e-02, -3.79616834e-02, -4.06346773e-02, -3.85919639e-02,
       -3.78490606e-02, -3.71552890e-02, -3.83987965e-02, -3.86067582e-02,
       -3.73055906e-02, -3.68953102e-02, -3.80305659e-02, -3.72825213e-02,
       -3.92088202e-02, -3.77858178e-02, -3.58322527e-02, -3.72530555e-02,
       -3.46911619e-02, -3.38231778e-02, -3.23161613e-02, -3.21918911e-02,
       -3.05329197e-02, -2.85034903e-02, -2.74386219e-02, -2.77427176e-02,
       -2.94859193e-02, -3.02480164e-02, -3.30190358e-02, -3.15905291e-02,
       -2.84432076e-02, -2.66104260e-02, -3.12824683e-02, -3.24917166e-02,
       -3.02982431e-02, -3.36521040e-02, -3.81549736e-02, -3.95224538e-02,
       -3.49353215e-02, -3.28846535e-02, -3.21756680e-02, -3.29497444e-02,
       -3.14266422e-02, -3.33369771e-02, -3.29869863e-02, -3.63844269e-02,
       -3.59501889e-02, -3.56222124e-02, -3.85983398e-02, -3.94228488e-02,
       -4.05945410e-02, -3.92177719e-02, -3.75704189e-02, -3.70137574e-02,
       -3.59055348e-02, -3.58662065e-02, -3.22298575e-02, -3.23332312e-02,
       -2.97053916e-02, -3.00357445e-02, -2.69307969e-02, -2.76470725e-02,
       -2.55997709e-02, -2.62774749e-02, -2.88235637e-02, -2.84899287e-02,
...
        4.68960151e-02,  4.61291708e-02,  4.75191651e-02,  4.48414036e-02,
        4.35055619e-02,  3.95616230e-02,  3.81319146e-02,  4.19703313e-02,
        4.07644184e-02,  3.75833302e-02,  3.81346376e-02,  3.67257666e-02,
        3.85689822e-02,  3.58255954e-02,  3.87826397e-02,  3.63567026e-02,
        3.76920989e-02,  3.71999179e-02,  3.92351141e-02,  3.84158789e-02,
        3.67394594e-02,  3.80376275e-02,  3.73408862e-02,  3.69631653e-02,
        3.70375349e-02,  3.65429838e-02,  3.52474070e-02,  3.84936764e-02,
        3.89783859e-02,  3.98229676e-02,  4.20480883e-02,  4.35959911e-02,
        4.25575228e-02,  4.26280674e-02,  4.41625799e-02,  4.68153329e-02,
        4.65814054e-02,  4.84182304e-02,  4.76861393e-02,  4.89737203e-02,
        4.70042633e-02,  4.66787679e-02,  4.50475369e-02,  4.50925558e-02,
        4.21787858e-02,  3.87463446e-02,  3.69870552e-02,  3.78328434e-02,
        3.72519643e-02,  3.79038241e-02,  3.78589533e-02,  3.79182504e-02,
        3.94812874e-02,  4.05942010e-02,  4.14256849e-02,  4.16150198e-02,
        4.00390524e-02,  3.98347028e-02,  4.25724223e-02,  4.11875764e-02,
        4.15165531e-02,  4.02136814e-02,  4.09656801e-02,  4.32152679e-02,
        4.38999878e-02,  4.65594048e-02,  4.69655246e-02,  4.77109392e-02,
        4.97469334e-02,  4.74873241e-02,  4.88236024e-02,  4.95368519e-02,
        5.27013975e-02,  5.39879564e-02,  5.42650585e-02,  5.65760609e-02,
        5.48986579e-02,  5.58048552e-02,  5.36677497e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Orbit Error - 2cm

In [33]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_orbiterror_2cm.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplie

In [34]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [35]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [36]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_orbiterror=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [37]:
# make new DataArray objects
gmsl_grids_orbiterror_2cm = xr.DataArray(ssh_weighted_mean_orbiterror, dims=['time'])
gmsl_grids_orbiterror_2cm = gmsl_grids_orbiterror_2cm.assign_coords({'time':grids.time})
gmsl_grids_orbiterror_2cm.name = 'gmsl_grids'
gmsl_grids_orbiterror_2cm

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.37895970e-02, -3.23054548e-02, -2.95603301e-02, -3.02106854e-02,
       -3.08561835e-02, -3.24769764e-02, -3.08535162e-02, -3.27602974e-02,
       -3.29589211e-02, -3.93843612e-02, -4.08876019e-02, -4.04694191e-02,
       -4.04407419e-02, -4.29523942e-02, -4.30253748e-02, -4.03058436e-02,
       -4.07961637e-02, -3.84377988e-02, -4.04239216e-02, -3.89623506e-02,
       -3.78493696e-02, -3.72454323e-02, -3.89393312e-02, -3.89046563e-02,
       -3.72541767e-02, -3.67539171e-02, -3.81893057e-02, -3.78406342e-02,
       -3.92228026e-02, -3.78714020e-02, -3.75217774e-02, -3.71399005e-02,
       -3.48789316e-02, -3.38817384e-02, -3.20432155e-02, -3.18110531e-02,
       -3.10122427e-02, -2.86882549e-02, -2.86354332e-02, -2.80831874e-02,
       -2.90177320e-02, -3.00463406e-02, -3.28974102e-02, -3.20776929e-02,
       -2.84724751e-02, -2.67304900e-02, -3.13584857e-02, -3.26301533e-02,
       -3.07825520e-02, -3.45921726e-02, -3.82866531e-02, -3.92415775e-02,
       -3.49847380e-02, -3.35316275e-02, -3.25532345e-02, -3.32616245e-02,
       -3.17804274e-02, -3.34885743e-02, -3.33494401e-02, -3.63491312e-02,
       -3.61617757e-02, -3.59776421e-02, -3.88605731e-02, -3.94804701e-02,
       -4.09277721e-02, -3.91872085e-02, -3.75403307e-02, -3.71531359e-02,
       -3.60765379e-02, -3.61463996e-02, -3.18680885e-02, -3.21104831e-02,
       -2.97400276e-02, -2.98301179e-02, -2.74990133e-02, -2.74630802e-02,
       -2.54086472e-02, -2.61977681e-02, -2.78560530e-02, -2.81215150e-02,
...
        4.68741392e-02,  4.64803214e-02,  4.74894137e-02,  4.54621699e-02,
        4.43196211e-02,  3.95482050e-02,  3.80326738e-02,  4.16539091e-02,
        4.09472924e-02,  3.74609386e-02,  3.81536394e-02,  3.66754699e-02,
        3.78622635e-02,  3.57501151e-02,  3.88445505e-02,  3.60095521e-02,
        3.82949720e-02,  3.71197990e-02,  3.87537594e-02,  3.86890530e-02,
        3.63176298e-02,  3.79647935e-02,  3.70217644e-02,  3.67343523e-02,
        3.75184381e-02,  3.65027490e-02,  3.50400124e-02,  3.80280065e-02,
        3.93429462e-02,  3.99494777e-02,  4.15377177e-02,  4.32224342e-02,
        4.26694385e-02,  4.21883642e-02,  4.39190316e-02,  4.65941761e-02,
        4.62172573e-02,  4.84922629e-02,  4.74575845e-02,  4.90036053e-02,
        4.71679064e-02,  4.68851028e-02,  4.52860913e-02,  4.54610219e-02,
        4.24889050e-02,  3.91935087e-02,  3.68208203e-02,  3.75055538e-02,
        3.75346025e-02,  3.74409710e-02,  3.74476447e-02,  3.76682873e-02,
        3.91736918e-02,  4.04598979e-02,  4.11175342e-02,  4.14045087e-02,
        4.01101958e-02,  3.98845581e-02,  4.24408499e-02,  4.12087827e-02,
        4.19139553e-02,  4.10680505e-02,  4.09624210e-02,  4.35533990e-02,
        4.44034945e-02,  4.64686110e-02,  4.61695203e-02,  4.74380709e-02,
        5.01709406e-02,  4.82217279e-02,  4.86768178e-02,  4.98652079e-02,
        5.24838702e-02,  5.35633568e-02,  5.43806346e-02,  5.58034068e-02,
        5.53358134e-02,  5.56999898e-02,  5.42804373e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### 3 errors: random noise + missing data + orbit error - 2cm

In [38]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_3errors_2cm.glob('*SSHA_gridded*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-25.n

In [39]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [40]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [41]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_3errors=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [42]:
# make new DataArray objects
gmsl_grids_3errors_2cm = xr.DataArray(ssh_weighted_mean_3errors, dims=['time'])
gmsl_grids_3errors_2cm = gmsl_grids_3errors_2cm.assign_coords({'time':grids.time})
gmsl_grids_3errors_2cm.name = 'gmsl_grids'
gmsl_grids_3errors_2cm

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([            nan,             nan, -2.99868547e-02, -3.07599721e-02,
       -3.19414910e-02, -3.31806179e-02, -3.23278269e-02, -3.34858451e-02,
       -3.29266791e-02, -3.89687528e-02, -4.05709643e-02, -3.96741734e-02,
       -4.04088674e-02, -4.21496600e-02, -4.30141251e-02, -3.95569416e-02,
       -4.02940088e-02, -3.72544610e-02, -4.01393365e-02, -3.83824780e-02,
       -3.75803856e-02, -3.71864623e-02, -3.84710918e-02, -3.82391590e-02,
       -3.71276428e-02, -3.68747275e-02, -3.76038848e-02, -3.78893576e-02,
       -3.92925889e-02, -3.73388962e-02, -3.51958716e-02, -3.74259215e-02,
       -3.45486769e-02, -3.38914349e-02, -3.27134739e-02, -3.23009299e-02,
       -3.02970092e-02, -2.83123395e-02, -2.74404337e-02, -2.78397596e-02,
       -2.93517094e-02, -3.00554784e-02, -3.32183526e-02, -3.14428503e-02,
       -2.88494680e-02, -2.66074581e-02, -3.14576133e-02, -3.27190715e-02,
       -3.02258283e-02, -3.36179597e-02, -3.79038871e-02, -3.97383243e-02,
       -3.45452444e-02, -3.30816718e-02, -3.18601213e-02, -3.28692532e-02,
       -3.11727728e-02, -3.37611145e-02, -3.30323566e-02, -3.63871120e-02,
       -3.63514083e-02, -3.57903299e-02, -3.83647844e-02, -3.93577810e-02,
       -4.06549716e-02, -3.90293008e-02, -3.76061174e-02, -3.69598635e-02,
       -3.59954909e-02, -3.60625282e-02, -3.28491958e-02, -3.21313389e-02,
       -2.92013243e-02, -2.98457181e-02, -2.69507798e-02, -2.81911228e-02,
       -2.54672575e-02, -2.64672273e-02, -2.88076657e-02, -2.86165369e-02,
...
        4.65351401e-02,  4.66184396e-02,  4.76644988e-02,  4.42709345e-02,
        4.35738572e-02,  3.93476245e-02,  3.81563641e-02,  4.23568544e-02,
        4.06114933e-02,  3.74082615e-02,  3.78684594e-02,  3.71065098e-02,
        3.82841633e-02,  3.57076502e-02,  3.85697964e-02,  3.64881376e-02,
        3.81621650e-02,  3.73968254e-02,  3.90018462e-02,  3.79957211e-02,
        3.68995136e-02,  3.71213181e-02,  3.69957334e-02,  3.71472902e-02,
        3.70593723e-02,  3.60582613e-02,  3.50155233e-02,  3.86872421e-02,
        3.90687168e-02,  4.03271573e-02,  4.24214144e-02,  4.33579419e-02,
        4.30387474e-02,  4.31733424e-02,  4.42412913e-02,  4.71049386e-02,
        4.67013517e-02,  4.81941574e-02,  4.78991853e-02,  4.92886993e-02,
        4.68560077e-02,  4.68264700e-02,  4.48611983e-02,  4.52561723e-02,
        4.23363338e-02,  3.82032076e-02,  3.69622295e-02,  3.82002619e-02,
        3.70543918e-02,  3.76479520e-02,  3.77618518e-02,  3.75681981e-02,
        3.99294472e-02,  4.07979546e-02,  4.12620310e-02,  4.13326917e-02,
        3.96039543e-02,  3.99409841e-02,  4.26313964e-02,  4.09964697e-02,
        4.15003460e-02,  4.02998569e-02,  4.01724735e-02,  4.27980228e-02,
        4.32495265e-02,  4.63445158e-02,  4.64150021e-02,  4.74866244e-02,
        4.96407017e-02,  4.76276491e-02,  4.85836443e-02,  4.91878833e-02,
        5.26212750e-02,  5.42103853e-02,  5.43613865e-02,  5.69123544e-02,
        5.50791771e-02,  5.62782092e-02,  5.36793876e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Save to Disk

In [43]:
fname = output_dir / ('GMSL_grids_19922017_noerror_100points_nosicapplied.nc')
gmsl_grids.to_netcdf(fname)

In [44]:
fname = output_dir / ('GMSL_grids_19922017_randomnoise_100points_nosicapplied.nc')
gmsl_grids_randomnoise.to_netcdf(fname)

In [45]:
fname = output_dir / ('GMSL_grids_19922017_missingdata_100points_nosicapplied.nc')
gmsl_grids_missingdata.to_netcdf(fname)

In [46]:
fname = output_dir / ('GMSL_grids_19922017_orbiterror_2cm_100points_nosicapplied.nc')
gmsl_grids_orbiterror_2cm.to_netcdf(fname)

In [47]:
fname = output_dir / ('GMSL_grids_19922017_3errors_2cm_100points_nosicapplied.nc')
gmsl_grids_3errors_2cm.to_netcdf(fname)